# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import time
import os
import csv

# Import API key
from api_keys import g_key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities_list = pd.read_csv("..\output_data\cities.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key // gkey not working? 
gmaps.configure(api_key=g_key)

In [6]:
locations = cities_list[["Lat", "Lng"]].astype(float)
humidity = cities_list["Humidity"].astype(float)
heatmap_humidity = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Create a combined map 
fig = gmaps.figure()
fig.add_layer(heatmap_humidity)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Fitting weather criteria is no hotter than 70 degrees fahrenheit and low humidity 
vacation_spots = cities_list.loc[(cities_list["Max Temp"] < 70) & (cities_list["Humidity"] < 30), :]
vacation_spots

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24
42,42,yellowknife,75,CA,1558378762,26,62.45,-114.38,55.40,8.05
156,156,narsaq,20,GL,1558378784,28,60.91,-46.05,57.20,9.17
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50
187,187,mackay,20,CA,1558378789,28,53.65,-115.58,59.00,20.80
230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20
282,282,slave lake,1,CA,1558378673,25,55.28,-114.77,62.60,17.22
299,299,jiuquan,0,CN,1558378844,14,39.73,98.49,59.64,5.50
324,324,sidney,1,US,1558378850,26,47.72,-104.16,61.00,14.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [28]:
# Store into variable named hotel_df.
hotel_df = pd.DataFrame(vacation_spots)

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24,
42,42,yellowknife,75,CA,1558378762,26,62.45,-114.38,55.40,8.05,
156,156,narsaq,20,GL,1558378784,28,60.91,-46.05,57.20,9.17,
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50,
187,187,mackay,20,CA,1558378789,28,53.65,-115.58,59.00,20.80,
230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87,
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20,
282,282,slave lake,1,CA,1558378673,25,55.28,-114.77,62.60,17.22,
299,299,jiuquan,0,CN,1558378844,14,39.73,98.49,59.64,5.50,
324,324,sidney,1,US,1558378850,26,47.72,-104.16,61.00,14.99,


In [41]:
# params dictionary to update each iteration
target_radius = 5000  
target_type = "Business"
target_search = "hotels" #terms?


# set up a parameters dictionary
params = {
    "radius": target_radius,
    "types": target_type,
    "keyword": target_search,
    "key": g_key
}

# loop through df, make an API call, according to parameters
for index, row in hotel_df.iterrows():
        # get lat, lng from df
        lat = row["Lat"]
        lng = row["Lng"]
        city = row["City"]
        # change location each iteration while leaving original params in place
        params["location"] = f"{lat},{lng}"
        print(f"Hotel found in {city}")
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        hotel_list = requests.get(base_url, params=params)
        # convert to json
        hotels = hotel_list.json()
        
        # If there is an error tell us no hotel found and continue
        try: 
              hotel_name = hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
              print(f"{hotel_name} is the nearest hotel in {city}.")
              print()
        
        except: 
              print(f"No hotel found near {city}.")
              print()


Hotel found in gobabis
Goba Lodge is the nearest hotel in gobabis.

Hotel found in yellowknife
The Explorer Hotel is the nearest hotel in yellowknife.

Hotel found in narsaq
Igdlo Guesthouse is the nearest hotel in narsaq.

Hotel found in yumen
Yumen Oriental Hotel is the nearest hotel in yumen.

Hotel found in mackay
Diamond Wood RV Resort is the nearest hotel in mackay.

Hotel found in moses lake
Best Western Plus Lake Front Hotel is the nearest hotel in moses lake.

Hotel found in hami
Hami Hotel is the nearest hotel in hami.

Hotel found in slave lake
Holiday Inn Express & Suites Slave Lake, an IHG Hotel is the nearest hotel in slave lake.

Hotel found in jiuquan
Jiuquan Hotel is the nearest hotel in jiuquan.

Hotel found in sidney
Holiday Inn Express & Suites Sidney, an IHG Hotel is the nearest hotel in sidney.

Hotel found in datong
Yunzhong Yi Traditional Inn is the nearest hotel in datong.

Hotel found in pampierstad
No hotel found near pampierstad.

Hotel found in el alto
Hote

In [42]:
hotel_df # with hotels added

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24,Goba Lodge
42,42,yellowknife,75,CA,1558378762,26,62.45,-114.38,55.40,8.05,The Explorer Hotel
156,156,narsaq,20,GL,1558378784,28,60.91,-46.05,57.20,9.17,Igdlo Guesthouse
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50,Yumen Oriental Hotel
187,187,mackay,20,CA,1558378789,28,53.65,-115.58,59.00,20.80,Diamond Wood RV Resort
230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87,Best Western Plus Lake Front Hotel
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20,Hami Hotel
282,282,slave lake,1,CA,1558378673,25,55.28,-114.77,62.60,17.22,"Holiday Inn Express & Suites Slave Lake, an IH..."
299,299,jiuquan,0,CN,1558378844,14,39.73,98.49,59.64,5.50,Jiuquan Hotel
324,324,sidney,1,US,1558378850,26,47.72,-104.16,61.00,14.99,"Holiday Inn Express & Suites Sidney, an IHG Hotel"


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [48]:
# Add marker layer ontop of heat map
# Create marker layer
marker_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"{hotel_name}" for hotel_name in hotel_df["Hotel Name"]]
)


fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))

In [49]:
# combine layers
fig = gmaps.figure()

fig.add_layer(heatmap_humidity)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))